In [3]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

# Load your CSV file from Google Drive
file_path = '/content/drive/MyDrive/kiwisaver_sample_wk9.csv'
data = pd.read_csv(file_path)

# Display the data
print(data.head())


                                       Name  \
0             AMP IL no E Fee NZ Share (RN)   
1        AMP KiwiSaver AMP Income Generator   
2           AMP KiwiSaver ANZ Balanced Plus   
3            AMP Prem PSS OnePath NZ Shares   
4  ANZ Default KiwiSaver Scheme Cnsrv(Dflt)   

                             Firm Name Branding Name  Holdings  \
0                            AMP Group           AMP        59   
1                            AMP Group           AMP       689   
2                            AMP Group           AMP      1544   
3                            AMP Group           AMP       125   
4  ANZ New Zealand Investments Limited           ANZ      1024   

                  Global Category  Morningstar Rating     Domicile  \
0              New Zealand Equity                 3.0  New Zealand   
1             Moderate Allocation                 NaN  New Zealand   
2           Aggressive Allocation                 3.0  New Zealand   
3  Australia & New Zealand Equity       

**Task 1: Inspecting your Data. read and disect your datafile "kiwisaver_sample_wk9.csv" to figure out list of all possible dirty data problems in your data.**

Lets see if we have any missing values or not

In [5]:
# Check for missing values in each column
missing_values = data.isnull().sum()

# Display columns with missing values and their counts
print(missing_values[missing_values > 0])


Morningstar Rating    36
Manager Name          43
Manager Tenure        43
Net Assets Date        1
dtype: int64


Lets see if there are any duplicates

In [6]:
# Check for duplicates in the dataset
duplicate_rows = data.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

Number of duplicate rows: 0


There are no duplicates.

Lets check for any special characters

In [8]:
import re
pattern = r'[^a-zA-Z0-9\s]'
for col in data.select_dtypes(include=['object']):
    special_chars = data[col].str.contains(pattern, na=False)
    print(f"Special characters in column {col}:\n", data[special_chars])


Special characters in column Name:
                                         Name  \
0              AMP IL no E Fee NZ Share (RN)   
4   ANZ Default KiwiSaver Scheme Cnsrv(Dflt)   
5      ANZ Default KiwiSaver Scheme-Balanced   
6   ANZ Default KiwiSaver Scheme-Balanced Gr   
7     ANZ Default KiwiSaver Scheme-Cnsrv Bal   
8        ANZ Default KiwiSaver Scheme-Growth   
9                     ANZ KiwiSaver-Balanced   
10             ANZ KiwiSaver-Balanced Growth   
11                ANZ KiwiSaver-Conservative   
12       ANZ KiwiSaver-Conservative Balanced   
13                      ANZ KiwiSaver-Growth   
16           ASB KiwiSaver Scheme's Balanced   
17    ASB KiwiSaver Scheme's Cnsrv (Default)   
18             ASB KiwiSaver Scheme's Growth   
19           ASB KiwiSaver Scheme's Moderate   
21      BNZ KiwiSaver Conservative (Default)   
34      Booster KiwiSaver Trans-Tasman Share   
35     FC KiwiSaver Scheme's Active Balanced   
36        FC KiwiSaver Scheme's Active Cnsrv   
37  

Task 1 answer: This dataset has issues of  missing values, inconsistent data types, hidden characters or spaces.

**Task 2: You must have noticed we have some data with missing "Morningstar Ratings", createa method and figure out a way to imputate the missing data**

In [9]:
def impute_rating(row):
    if pd.isnull(row["Morningstar Rating"]):
        mode_rating = data[data["Equity Style"] == row["Equity Style"]]["Morningstar Rating"].mode()
        return mode_rating[0] if not mode_rating.empty else row["Morningstar Rating"]
    else:
        return row["Morningstar Rating"]

data["Morningstar Rating"] = data.apply(impute_rating, axis=1)

overall_mode_rating = data["Morningstar Rating"].mode()[0]
data["Morningstar Rating"].fillna(overall_mode_rating, inplace=True)

print(data["Morningstar Rating"].isnull().sum())


0


<ipython-input-9-be26a24a86b5>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Morningstar Rating"].fillna(overall_mode_rating, inplace=True)


Lets see if we have perfomed task 2 successfully

In [11]:
# Check if there are any missing values left in "Morningstar Rating"
missing_ratings = data["Morningstar Rating"].isnull().sum()
print("Number of missing values in 'Morningstar Rating':", missing_ratings)


Number of missing values in 'Morningstar Rating': 0


Lets see the Morningstar Ratings now:

In [12]:
print(data[["Equity Style", "Morningstar Rating"]].head(10))  # Adjust as needed to see more rows

   Equity Style  Morningstar Rating
0    Mid Growth                 3.0
1     Mid Blend                 4.0
2  Large Growth                 3.0
3    Mid Growth                 3.0
4    Mid Growth                 4.0
5  Large Growth                 3.0
6  Large Growth                 3.0
7  Large Growth                 4.0
8  Large Growth                 5.0
9  Large Growth                 4.0


**Task 3: List all the unique "Equity Style"? What are the two most used and least used Equity Style?**

To list unique equity style values:

In [13]:
unique_equity_styles = data["Equity Style"].unique()
print("Unique Equity Styles:", unique_equity_styles)

Unique Equity Styles: ['Mid Growth' 'Mid Blend' 'Large Growth' 'Large Blend' 'Mid Value']


In [14]:
# 2. Count occurrences of each "Equity Style"
equity_style_counts = data["Equity Style"].value_counts()


Lets see most used styles:

In [17]:
most_used_styles = equity_style_counts.head(2)
print("Most used Equity Styles:\n", most_used_styles)

Most used Equity Styles:
 Equity Style
Large Growth    33
Large Blend     28
Name: count, dtype: int64


Lets see the least used Equity Styles (bottom 2):

In [16]:
least_used_styles = equity_style_counts.tail(2)
print("Least used Equity Styles:\n", least_used_styles)

Least used Equity Styles:
 Equity Style
Mid Blend    3
Mid Value    2
Name: count, dtype: int64


**Task 4:Mapping your data: Figure out a way to map the values of "Morningstar Rating" form [5,4,3,2,1] to ['Awsome','Good','Average','bad','Awful']**

In [19]:
rating_mapping = {5: 'Awesome', 4: 'Good', 3: 'Average', 2: 'Bad', 1: 'Awful'}

Lets apply the mapping to the "Morningstar Rating" column

In [20]:
data["Morningstar Rating Label"] = data["Morningstar Rating"].map(rating_mapping)

Lets display updated form:

In [22]:
print(data[["Morningstar Rating", "Morningstar Rating Label"]].head(10))

   Morningstar Rating Morningstar Rating Label
0                 3.0                  Average
1                 4.0                     Good
2                 3.0                  Average
3                 3.0                  Average
4                 4.0                     Good
5                 3.0                  Average
6                 3.0                  Average
7                 4.0                     Good
8                 5.0                  Awesome
9                 4.0                     Good


**Task 5: Data Aggregation using GroupBy. Using the datafile "Kiwisaver_sample_wk9.csv", group the Funds based on Branding Name and Equity Style. Report the average Holdings and Net Assets Size of the groups that you just created.**

Lets group by branding name and equity style:

In [23]:
grouped_data = data.groupby(["Branding Name", "Equity Style"])

Lets find mean for holdings and net asset size:

In [24]:
aggregated_means = grouped_data[["Holdings", "Net Assets Size"]].mean()

Display the data now:

In [25]:
print(aggregated_means)

                                     Holdings  Net Assets Size
Branding Name       Equity Style                              
AMP                 Large Growth  1544.000000     1.466660e+05
                    Mid Blend      689.000000     8.619230e+02
                    Mid Growth      92.000000     6.918946e+04
ANZ                 Large Growth  1543.111111     5.191668e+05
                    Mid Growth     438.666667     2.460913e+05
ASB                 Large Blend   1319.500000     1.191800e+06
                    Large Growth  1222.333333     5.645363e+03
Aon                 Large Blend    265.000000     1.911613e+04
                    Mid Growth     384.000000     8.220035e+04
Bank of New Zealand Large Blend   2456.800000     1.425594e+05
Fisher Funds        Large Growth   209.500000     6.073430e+05
Forsyth Barr        Large Growth   153.000000     1.142776e+04
Generate            Mid Blend       56.000000     1.074962e+05
                    Mid Growth     108.000000     5.560

Now lets display the complete updated file now:

In [26]:
print(data.head())

                                       Name  \
0             AMP IL no E Fee NZ Share (RN)   
1        AMP KiwiSaver AMP Income Generator   
2           AMP KiwiSaver ANZ Balanced Plus   
3            AMP Prem PSS OnePath NZ Shares   
4  ANZ Default KiwiSaver Scheme Cnsrv(Dflt)   

                             Firm Name Branding Name  Holdings  \
0                            AMP Group           AMP        59   
1                            AMP Group           AMP       689   
2                            AMP Group           AMP      1544   
3                            AMP Group           AMP       125   
4  ANZ New Zealand Investments Limited           ANZ      1024   

                  Global Category  Morningstar Rating     Domicile  \
0              New Zealand Equity                 3.0  New Zealand   
1             Moderate Allocation                 4.0  New Zealand   
2           Aggressive Allocation                 3.0  New Zealand   
3  Australia & New Zealand Equity       